In [2]:
#from boundingbox_suppression import *
import numpy as np
import cv2

In [1]:
path = '/home/dick/ros_test/catkin_ws/src/camera/data/realsense/for_test/points/'
file_name = '31_rgb.png'
img = cv2.imread(path + file_name)
ori = img.copy()
#detected_img, roi_list = detect_color(ori)
cv2.imshow('hi', ori)
cv2.waitKey(1)

NameError: name 'cv2' is not defined

In [11]:
cv2.imshow("Original", img)
# cv2.imshow("After NMS", out)
cv2.waitKey(0) 

-1

In [3]:
def getRed(img):
	lower_bound_0 = np.array([0, 100, 230]) 
	upper_bound_0 = np.array([10, 255, 255])
	lower_bound_1 = np.array([170, 100, 50]) 
	upper_bound_1 = np.array([180, 255, 255])
	hsv_img   = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
	mask_0 = cv2.inRange(hsv_img, lower_bound_0, upper_bound_0)
	mask_1 = cv2.inRange(hsv_img, lower_bound_1, upper_bound_1)
	mask  = cv2.bitwise_or(mask_0, mask_1)
	return mask

def getGreen(img):
	lower_bound_0 = np.array([65,60,60]) 
	upper_bound_0 = np.array([85,255,255])
	hsv_img   = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
	mask = cv2.inRange(hsv_img, lower_bound_0, upper_bound_0)
	return mask

def getBlue(img):
	lower_bound_0 = np.array([70,60,20]) 
	upper_bound_0 = np.array([140,255,255])
	hsv_img   = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
	mask = cv2.inRange(hsv_img, lower_bound_0, upper_bound_0)
	return mask


def detect_color(img):
	canvas = img.copy()
	#print(canvas.shape)
	dim = canvas.shape

	red_mask   = getRed(canvas)
	contours_red, hierarchy = cv2.findContours(red_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
	green_mask = getGreen(canvas)
	contours_green, hierarchy = cv2.findContours(green_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        blue_mask = getBlue(canvas)
	contours_blue, hierarchy = cv2.findContours(blue_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

	roi_list = []
	for cnt in contours_red:
		x, y, w, h = cv2.boundingRect(cnt)
                #if((w * h) > 500*4):
		if(((w * h) > 500*4) & ((w * h) < 57600)):
			cv2.rectangle(canvas, (x - 10, y - 10), (x + w + 10, y + h + 10), (0, 255, 0), 2)
			x1 = x - 10 if (x - 10) > 0 else 0
			y1 = y - 10 if (y - 10) > 0 else 0
			x2 = x+w+10 if (x+w+10) < dim[1] else dim[1]-1
			y2 = y+h+10 if (y+h+10) < dim[0] else dim[0]-1

			roi_list.append((x1, y1, x2, y2))
			cv2.putText(canvas, 'Red', (x - 10, y - 14), cv2.FONT_HERSHEY_DUPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)

	for cnt in contours_green:
		x, y, w, h = cv2.boundingRect(cnt)
		#if((w * h) > 500*4):
                if(((w * h) > 500*4) & ((w * h) < 57600)):
			cv2.rectangle(canvas, (x - 10, y - 10), (x + w + 10, y + h + 10), (255, 0, 0), 2)
			x1 = x - 10 if (x - 10) > 0 else 0
			y1 = y - 10 if (y - 10) > 0 else 0
			x2 = x+w+10 if (x+w+10) < dim[1] else dim[1]-1
			y2 = y+h+10 if (y+h+10) < dim[0] else dim[0]-1

			roi_list.append((x1, y1, x2, y2))
			cv2.putText(canvas, 'Green', (x - 10, y - 14), cv2.FONT_HERSHEY_DUPLEX, 0.5, (255, 0, 0), 1, cv2.LINE_AA)

        for cnt in contours_blue:
		x, y, w, h = cv2.boundingRect(cnt)
		#if((w * h) > 500*4):
                if(((w * h) > 500*4) & ((w * h) < 57600)):
			cv2.rectangle(canvas, (x - 10, y - 10), (x + w + 10, y + h + 10), (255, 0, 0), 2)
			x1 = x - 10 if (x - 10) > 0 else 0
			y1 = y - 10 if (y - 10) > 0 else 0
			x2 = x+w+10 if (x+w+10) < dim[1] else dim[1]-1
			y2 = y+h+10 if (y+h+10) < dim[0] else dim[0]-1

			roi_list.append((x1, y1, x2, y2))
			cv2.putText(canvas, 'Blue', (x - 10, y - 14), cv2.FONT_HERSHEY_DUPLEX, 0.5, (255, 0, 0), 1, cv2.LINE_AA)

	return canvas, np.array(roi_list)

In [4]:
def non_max_suppression_slow(boxes, overlapThresh):
    # if there are no boxes, return an empty list
    if len(boxes) == 0:
        return []
    # initialize the list of picked indexes
    pick = []
    # grab the coordinates of the bounding boxes
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]
    # compute the area of the bounding boxes and sort the bounding
    # boxes by the bottom-right y-coordinate of the bounding box
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)

        # keep looping while some indexes still remain in the indexes list
    print('in func')
    while len(idxs) > 0:
        # grab the last index in the indexes list, add the index
        # value to the list of picked indexes, then initialize
        # the suppression list (i.e. indexes that will be deleted)
        # using the last index
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
        suppress = [last]
        
        # loop over all indexes in the indexes list
        for pos in range(0, last):
            # grab the current index
            j = idxs[pos]
            # find the largest (x, y) coordinates for the start of
            # the bounding box and the smallest (x, y) coordinates
            # for the end of the bounding box
            xx1 = max(x1[i], x1[j])
            yy1 = max(y1[i], y1[j])
            xx2 = min(x2[i], x2[j])
            yy2 = min(y2[i], y2[j])
            # compute the width and height of the bounding box
            w = max(0, xx2 - xx1 + 1)
            h = max(0, yy2 - yy1 + 1)
            # compute the ratio of overlap between the computed
            # bounding box and the bounding box in the area list
            overlap = float(w * h) / area[j]
            # if there is sufficient overlap, suppress the
            # current bounding box
            if overlap > overlapThresh:
                suppress.append(pos)
        # delete all indexes from the index list that are in the
        # suppression list
        idxs = np.delete(idxs, suppress)
# return only the bounding boxes that were picked
    return boxes[pick]